## 1. Data Preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
dataset_path = "/kaggle/input/dataset-x-fake-profile-zip/dataset_x_fake_profile/"
print(f"📂 Dataset path: {dataset_path}")



In [ ]:
import os

dataset_path = "/kaggle/input/dataset-x-fake-profile-zip/dataset_x_fake_profile/"
base_len = len(dataset_path)
total_images = 0

for root, dirs, files in os.walk(dataset_path):
    num_images = len([f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    if num_images > 0:
        short_path = root[base_len:]
        total_images += num_images
        print(f"📁 Folder: {short_path} - 🖼️ Number of images: {num_images}")

print("\n📊 Total number of images in the dataset:", total_images)


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

input_root = "/kaggle/input/dataset-x-fake-profile-zip/dataset_x_fake_profile"
output_root = "/kaggle/working/dataset_x_normalized"

def min_max_normalization(img):
    img = img.astype(np.float32)
    min_val = np.min(img, axis=(0, 1), keepdims=True)
    max_val = np.max(img, axis=(0, 1), keepdims=True)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1  # Avoid division by zero
    normalized_img = (img - min_val) / range_val * 255
    return np.clip(normalized_img, 0, 255).astype(np.uint8)

def process_image(args):
    input_path, output_path, normalization_func = args
    img = cv2.imread(input_path)
    if img is None:
        return False  # Failed to load image

    norm_img = normalization_func(img)
    if len(norm_img.shape) == 2:
        norm_img = cv2.cvtColor(norm_img, cv2.COLOR_GRAY2BGR)

    cv2.imwrite(output_path, norm_img)
    return True

def process_and_save_images(normalization_func, technique_name):
    for split in ["train", "test", "validation"]:
        for category in ["verified", "real", "cyborg", "bot"]:
            input_dir = os.path.join(input_root, split, category)
            output_dir = os.path.join(output_root, technique_name, split, category)
            os.makedirs(output_dir, exist_ok=True)

            images = os.listdir(input_dir)
            total_images = len(images)

            args_list = []
            for img_name in images:
                input_path = os.path.join(input_dir, img_name)
                output_path = os.path.join(output_dir, img_name)
                args_list.append((input_path, output_path, normalization_func))

            with Pool(processes=cpu_count()) as pool:
                results = list(tqdm(pool.imap_unordered(process_image, args_list), total=total_images,
                                    desc=f"🔄 {split}/{category} ({technique_name})", unit="image"))

    print(f"✅ {technique_name} normalization applied and images saved successfully!")

process_and_save_images(min_max_normalization, "min_max")


In [ ]:
import os
import torch

base_data_dir = "/kaggle/working/dataset_x_normalized/min_max" 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

splits = ["train", "validation", "test"]
categories = ["verified", "real", "cyborg", "bot"]

for split in splits:
    for category in categories:
        path = os.path.join(base_data_dir, split, category)
        if not os.path.exists(path):
            print(f"⚠️ Warning: Folder {path} does not exist!")
        else:
            print(f"📂 Path exists: {path}")

print(f"\n🖥️ Device in use: {device}")

print(f"📂 Test data path: {os.path.join(base_data_dir, 'test')}")


## 2. Image Transformations


In [ ]:
from torchvision import transforms

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.4),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Image transformations set successfully!")



## 3. Data Loading

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import os

base_data_dir = "/kaggle/working/dataset_x_normalized/min_max"

train_path = os.path.join(base_data_dir, "train")
validation_path = os.path.join(base_data_dir, "validation")
test_path = os.path.join(base_data_dir, "test")

assert os.path.exists(train_path), f"❌ Train path does not exist: {train_path}"
assert os.path.exists(validation_path), f"❌ Validation path does not exist: {validation_path}"
assert os.path.exists(test_path), f"❌ Test path does not exist: {test_path}"

train_dataset = ImageFolder(root=train_path, transform=transform_train)
validation_dataset = ImageFolder(root=validation_path, transform=transform_val_test)
test_dataset = ImageFolder(root=test_path, transform=transform_val_test)

print("✅ Data loaded successfully!")
print(f"🔹 Number of training images: {len(train_dataset)}")
print(f"🔹 Number of validation images: {len(validation_dataset)}")
print(f"🔹 Number of test images: {len(test_dataset)}")

use_gpu = torch.cuda.is_available()
num_workers = min(8, os.cpu_count() // 2) if use_gpu else min(4, os.cpu_count() // 4)
pin_memory = use_gpu
prefetch_factor = 4 if num_workers > 0 else None

train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True,
    num_workers=num_workers, pin_memory=pin_memory,
    persistent_workers=(use_gpu and num_workers > 0), prefetch_factor=prefetch_factor,
    drop_last=False
)

validation_loader = DataLoader(
    validation_dataset, batch_size=32, shuffle=False,
    num_workers=num_workers, pin_memory=pin_memory,
    persistent_workers=(use_gpu and num_workers > 0), prefetch_factor=prefetch_factor
)

test_loader = DataLoader(
    test_dataset, batch_size=32, shuffle=False,
    num_workers=num_workers, pin_memory=pin_memory,
    persistent_workers=(use_gpu and num_workers > 0), prefetch_factor=prefetch_factor
)

print("🚀 DataLoaders are ready to use!")


## 4. Model Architecture

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

weights_path = "/kaggle/input/resnet50-11ad3fa6-pth/resnet50-11ad3fa6.pth"
model = models.resnet50()
model.load_state_dict(torch.load(weights_path, map_location="cuda" if torch.cuda.is_available() else "cpu"))

for param in model.parameters():
    param.requires_grad = False

for layer in [model.layer2, model.layer3, model.layer4]:
    for param in layer.parameters():
        param.requires_grad = True

num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(num_features, 1024),
    nn.BatchNorm1d(1024),
    nn.LeakyReLU(0.1, inplace=True),
    nn.Dropout(0.25),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.LeakyReLU(0.1, inplace=True),
    nn.Dropout(0.15),
    nn.Linear(512, 4)
)

for param in model.fc.parameters():
    param.requires_grad = True

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Number of trainable parameters: {trainable_params:,}")
print(model.fc)
print("✅ Model is successfully prepared for training 🎯")


## 5. Optimizer & Scheduler


In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                        lr=3e-4, weight_decay=1e-4, eps=1e-8)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.003,
    steps_per_epoch=len(train_loader),
    epochs=10,
    pct_start=0.2,
    anneal_strategy='cos',
    div_factor=25,
    final_div_factor=1000
)


## 7. Final Training


### 7.1 Main Training Function

In [ ]:
from tqdm import tqdm
import gc
import torch

def train_one_epoch(model, train_loader, optimizer, criterion, device, epoch, start_batch=0):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    total_batches = len(train_loader)

    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    progress_bar = tqdm(enumerate(train_loader), total=total_batches, desc=f"🚀 Epoch {epoch}", unit="batch", leave=False)

    for batch_idx, (images, labels) in progress_bar:
        if batch_idx < start_batch:
            continue

        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast() if scaler else torch.enable_grad():
            outputs = model(images)
            loss = criterion(outputs, labels)

        if scaler:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
        else:
            loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        if scaler:
            scaler.step(optimizer)
            scaler.update()
        else:
            optimizer.step()

        running_loss += loss.item() * images.size(0)
        correct += outputs.argmax(dim=1).eq(labels).sum().item()
        total += labels.size(0)

        avg_loss = running_loss / total if total > 0 else 0
        accuracy = 100.0 * correct / total if total > 0 else 0
        progress_bar.set_postfix(loss=avg_loss, acc=f"{accuracy:.2f}%", batch=batch_idx + 1)

        # 🧹 إدارة الذاكرة
        if torch.cuda.is_available() and batch_idx % 10 == 0:
            del images, labels, outputs, loss
            torch.cuda.empty_cache()
            gc.collect()

    avg_loss = running_loss / total if total > 0 else 0
    accuracy = 100.0 * correct / total if total > 0 else 0

    print(f"✅ Training completed for Epoch {epoch} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return avg_loss, accuracy


## 8. Evaluation & Logging

In [ ]:
import os
import gc
from datetime import datetime
import torch

def train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, num_epochs, base_save_path, start_epoch=1, early_stopping_patience=5):
    patience_counter = 0
    best_validation_loss = float("inf")

    results_dir = os.path.join(base_save_path)
    os.makedirs(results_dir, exist_ok=True)

    results_file = os.path.join(results_dir, "training_results.txt")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with open(results_file, "a", encoding="utf-8") as f:
        if os.stat(results_file).st_size == 0:
            f.write("📌 Training Log\n" + "=" * 50 + "\n")

    for epoch in range(start_epoch, num_epochs + 1):
        start_time = datetime.now()
        print(f"\n🔥 Training Epoch {epoch}/{num_epochs} ...")

        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)

        with torch.no_grad():
            validation_loss, validation_acc = validate_model(model, validation_loader, criterion, device)

        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(validation_loss)
        else:
            scheduler.step()

        elapsed_time = (datetime.now() - start_time).total_seconds()
        hours, remainder = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        elapsed_str = f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"

        best_model_path = os.path.join(base_save_path, "best_model.pth")
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"🏆 Best model saved at: {best_model_path}")
        else:
            patience_counter += 1
            print(f"⚠️ No improvement for {patience_counter} consecutive epochs (Patience: {early_stopping_patience})")

            if patience_counter >= early_stopping_patience:
                print("🛑 Early stopping triggered due to no improvement.")
                break

        with open(results_file, "a", encoding="utf-8") as f:
            f.write(f"\n🗓 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Epoch {epoch}/{num_epochs}\n")
            f.write("=" * 50 + "\n")
            f.write(f"⏳ Elapsed Time: {elapsed_str}\n")
            f.write(f"🎯 Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%\n")
            f.write(f"✅ Validation: Loss = {validation_loss:.4f}, Accuracy = {validation_acc:.2f}%\n")
            f.write("-" * 50 + "\n")

        gc.collect()
        torch.cuda.empty_cache()

    print(f"✅ Training completed successfully! Results saved at: {results_file}")


In [ ]:
print(f"Number of samples in train_dataset: {len(train_dataset)}")
print(f"Number of batches in train_loader: {len(train_loader)}")


In [ ]:
print(f"Batch size used in train_loader: {train_loader.batch_size}")


### 8.1 Validation with TTA

In [ ]:
import torch
from tqdm import tqdm

def tta_predictions(model, images):
    augmentations = [
        images,
        torch.flip(images, [3]),
        torch.rot90(images, 1, [2, 3]),
        torch.flip(images, [2]),
    ]

    with torch.no_grad(), torch.cuda.amp.autocast():
        outputs = torch.cat([model(aug) for aug in augmentations]).view(len(augmentations), -1, 4).mean(dim=0)
    return outputs

def validate_model(model, validation_loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    progress_bar = tqdm(validation_loader, desc="🧐 Validation", unit="batch", leave=False)

    with torch.no_grad():
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            outputs = tta_predictions(model, images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            correct += outputs.argmax(dim=1).eq(labels).sum().item()
            total += labels.size(0)

            avg_loss = running_loss / total if total > 0 else 0
            accuracy = 100.0 * correct / total if total > 0 else 0
            progress_bar.set_postfix(loss=avg_loss, acc=f"{accuracy:.2f}%")

    avg_loss = running_loss / total if total > 0 else 0
    accuracy = 100.0 * correct / total if total > 0 else 0

    print(f"\n✅ Validation Results:\n"
          f"🔹 Average Loss: {avg_loss:.4f}\n"
          f"🔹 Accuracy: {accuracy:.2f}%\n")

    return avg_loss, accuracy


In [ ]:
print(f"📦 Number of batches available in training: {len(train_loader)}")


### 7.5 Execute Final Training

In [ ]:
import torch
import os

base_save_path = "/kaggle/working/"
best_model_path = os.path.join(base_save_path, "best_model.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_epoch = 1
num_epochs = 10  # 🔧 عدد Epochs الكلي اللي تبي تدرب عليه

# ✅ تحميل أفضل نموذج محفوظ إن وجد
if os.path.exists(best_model_path):
    print("✅ Loading best saved model...")
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.to(device)

    training_log = os.path.join(base_save_path, "training_results.txt")
    if os.path.exists(training_log):
        with open(training_log, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for line in reversed(lines):
                if "Epoch" in line and "/" in line:
                    start_epoch = int(line.split(" ")[1].split("/")[0]) + 1
                    break

    print(f"🚀 Starting training from Epoch {start_epoch} using the best saved model.")

# ✅ بدء من الصفر إذا لم يوجد أي نموذج محفوظ
else:
    print("❌ No saved model found! Starting training from scratch.")
    model.to(device)

# ✅ إعداد Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-3)

# ✅ إعداد Scheduler باستخدام OneCycleLR
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.003,
    steps_per_epoch=len(train_loader),
    epochs=num_epochs - start_epoch + 1,  # ⏳ حسب ما تبقى من التدريب
    pct_start=0.2,
    anneal_strategy='cos',
    div_factor=25,
    final_div_factor=1000
)

torch.cuda.empty_cache()

# ✅ بدء التدريب
train_model(
    model,
    train_loader,
    validation_loader,
    criterion,
    optimizer,
    scheduler,
    num_epochs,
    base_save_path,
    start_epoch
)


## 9. Final Testing

### 9.1 Evaluate on Test Set

In [ ]:

import os
import shutil
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from tqdm import tqdm
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix



In [ ]:
from collections import Counter

label_counts = Counter([label for _, label in test_dataset.samples])
print("📊 Test dataset image distribution:")
for class_name, count in zip(test_dataset.classes, label_counts.values()):
    print(f"📁 {class_name}: {count} images")



In [ ]:
def show_images(dataset, num_images=6):
    fig, axes = plt.subplots(1, num_images, figsize=(12, 4))
    for i in range(num_images):
        img, label = dataset[i]
        img = img.permute(1, 2, 0).numpy() * 0.5 + 0.5  # فك عملية التطبيع
        axes[i].imshow(img)
        axes[i].set_title(dataset.classes[label])
        axes[i].axis("off")
    plt.tight_layout()
    plt.show()

show_images(test_dataset)


In [ ]:
import os
from datetime import datetime

def test_model(model, test_loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            correct += outputs.argmax(dim=1).eq(labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total if total > 0 else 0
    accuracy = 100.0 * correct / total if total > 0 else 0
    print(f"🎯 اختبار النموذج: Loss = {avg_loss:.4f}, دقة = {accuracy:.2f}%")

    return avg_loss, accuracy

# تنفيذ اختبار النموذج
test_loss, test_acc = test_model(model, test_loader, criterion, device)

# تحديد مسار حفظ النتائج
accuracy_path = "/kaggle/working/accuracy0.txt"
os.makedirs(os.path.dirname(accuracy_path), exist_ok=True)  # التأكد من وجود المجلد

# تسجيل النتيجة مع التاريخ والوقت
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open(accuracy_path, "a", encoding="utf-8") as f:
    f.write(f"[{timestamp}] Accuracy: {test_acc:.2f}%, Loss: {test_loss:.4f}\n")



### 9.3 Classification Report

In [ ]:
import os
import torch
from sklearn.metrics import classification_report
from datetime import datetime

true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predictions = outputs.argmax(dim=1).cpu().numpy()

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predictions)

report = classification_report(true_labels, predicted_labels, target_names=["Class 0", "Class 1", "Class 2", "Class 3"])
print("📄 Classification Report:\n", report)

report_path = "/kaggle/working/classification_report.txt"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(f"📌 Classification Report - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 50 + "\n")
    f.write(report)
    f.write("\n" + "=" * 50 + "\n")

print(f"✅ Classification report saved at: {report_path}")



### 9.4  confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(true_labels, predicted_labels)

xlabel = "Predictions"
ylabel = "True Values"
title = "Confusion Matrix"

plt.figure(figsize=(7, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=True,
            xticklabels=["Class 0", "Class 1", "Class 2", "Class 3"],
            yticklabels=["Class 0", "Class 1", "Class 2", "Class 3"])

plt.xlabel(xlabel, fontsize=14, labelpad=10, fontweight="bold")
plt.ylabel(ylabel, fontsize=14, labelpad=10, fontweight="bold")
plt.xticks(rotation=0, fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.title(title, fontsize=16, pad=15, fontweight="bold")

conf_matrix_path = "/kaggle/working/confusion_matrix.png"
plt.savefig(conf_matrix_path, dpi=300, bbox_inches="tight")

#print(f"✅ The confusion matrix has been saved at: {conf_matrix_path}")
plt.show()  
